# Publishing Feedback

### Setup

In [ ]:
# You can set them inline
import os
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"

In [ ]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

### Adding Feedback to a Run

You can add feedback to an existing Run programmatically as long as you know the run_id. Let's grab a run_id from the LangSmith UI and add it here.

In [ ]:
run_id = ""

Now, let's add some continuous feedback

In [ ]:
from langsmith import Client

client = Client()

client.create_feedback(
  run_id,
  key="sample-continuous",
  score=7.0,
  comment="This is a sample piece of continuous feedback",
)

And now, let's add some categorical feedback too!

In [ ]:
from langsmith import Client

client = Client()

client.create_feedback(
  run_id,
  key="sample-categorical",
  value="no",
  comment="This is a sample piece of categorical feedback",
)

### Pre-Generating Run IDs for Feedback

![Generate_run_id](../../images/generate_run_id.png)

Using LangChain, we offer the ability to pre-generate and define run IDs, before your code is invoked and the run ID is generated. With this functionality, you're able to access your run ID before initial generation, which can be useful for actions like sending feedback. The example below demonstrates this.

In [ ]:
import uuid

pre_defined_run_id = uuid.uuid4()
pre_defined_run_id

In [ ]:
from langsmith import traceable

@traceable
def foo():
    return "This is a sample Run!"

We are passing in a config with our function call through `langsmith_extra` that contains our pre-defined run_id

In [ ]:
foo(langsmith_extra={"run_id": pre_defined_run_id})

Now we can directly create feedback on this run!

In [ ]:
from langsmith import Client

client = Client()

client.create_feedback(pre_defined_run_id, "user_feedback", score=1)

### Pre-signed Feedback URLs

![presigned url](../../images/presigned_url.png)


This can also be helpful for pre-signed feedback URLs. You would want to use these when you can't expose API keys or other secrets to the client, e.g. in a web application. Using a pre-determined run_id LangSmith has an endpoint create_presigned_feedback_token which will create a URL for sending feedback, without the use of secrets required.

In [ ]:
pre_signed_url_id = uuid.uuid4()
pre_signed_url_id

In [ ]:
pre_signed_url = client.create_presigned_feedback_token(pre_signed_url_id, "user_presigned_feedback")

print(pre_signed_url)

Here, we can see that even though we haven't created a run yet, we're still able to generate the feedback URL.

Now, let's invoke our chain so the run with that ID is created:

In [ ]:
foo(langsmith_extra={"run_id": pre_signed_url_id})

Then, once our run is created, we can use the feedback URL to send feedback:

In [ ]:
import requests

url_with_score = f"{pre_signed_url.url}?score=1"

response = requests.get(url_with_score)

if response.status_code >= 200 and response.status_code < 300:
    print("Feedback submitted successfully!")
else:
    print("Feedback submission failed!")